### LightGBM으로 multiobjective 구현

In [110]:
import src.datasets as datasets
import src.surrogate as surrogate
import src.search as search

import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.model_selection import train_test_split


In [25]:
import pandas as pd
df = pd.read_csv('./data/concrete_processed.csv')
df

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,0.089726,0.619702,0.000000,0.757426,0.000000,0.496512,0.422059,0.000000,0.357597
1,0.152740,0.123356,0.621189,0.309901,0.488688,0.813372,0.562353,-0.666667,0.274815
2,0.337900,0.000000,0.478261,0.598020,0.248869,0.453198,0.753529,0.000000,0.348904
3,0.374429,0.333236,0.000000,1.000000,0.000000,0.380814,0.191176,0.000000,0.564681
4,0.120548,0.536101,0.000000,0.656436,0.411765,0.716279,0.269706,0.000000,0.207084
...,...,...,...,...,...,...,...,...,...
921,0.089726,0.619702,0.000000,0.757426,0.000000,0.496512,0.422059,-1.000000,0.104580
922,0.445662,0.000000,0.587206,0.473267,0.429864,0.644767,0.436765,-1.190476,0.254055
923,0.075342,0.000000,0.829585,0.524752,0.452489,0.465116,0.588235,0.000000,0.142208
924,0.398174,0.339082,0.451274,0.520792,0.402715,0.200872,0.480294,0.000000,0.544310


In [64]:
# target_dict = {9:'strength', 0:'cement', 3: 'water'}

In [67]:
# target_dict.values()

dict_values(['strength', 'cement', 'water'])

In [92]:
target_cols = ['strength', 'cement', 'water']

In [93]:
X = df.drop(columns=target_cols)
y = df[target_cols]    

In [94]:
X

,slag,ash,superplastic,coarseagg,fineagg,age
0,0.619702,0.000000,0.000000,0.496512,0.422059,0.000000
1,0.123356,0.621189,0.488688,0.813372,0.562353,-0.666667
2,0.000000,0.478261,0.248869,0.453198,0.753529,0.000000
3,0.333236,0.000000,0.000000,0.380814,0.191176,0.000000
4,0.536101,0.000000,0.411765,0.716279,0.269706,0.000000
...,...,...,...,...,...,...
921,0.619702,0.000000,0.000000,0.496512,0.422059,-1.000000
922,0.000000,0.587206,0.429864,0.644767,0.436765,-1.190476
923,0.000000,0.829585,0.452489,0.465116,0.588235,0.000000
924,0.339082,0.451274,0.402715,0.200872,0.480294,0.000000


In [95]:
y

,strength,cement,water
0,0.357597,0.089726,0.757426
1,0.274815,0.152740,0.309901
2,0.348904,0.337900,0.598020
3,0.564681,0.374429,1.000000
4,0.207084,0.120548,0.656436
...,...,...,...
921,0.104580,0.089726,0.757426
922,0.254055,0.445662,0.473267
923,0.142208,0.075342,0.524752
924,0.544310,0.398174,0.520792


In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True

    
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape: (740, 6)
y_train shape: (740, 3)
X_test shape:  (186, 6)
y_test shape:  (186, 3)


In [28]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_train.shape, type(X_train))
print(y_train.shape, type(y_train))
print(X_test.shape, type(X_test))
print(y_test.shape, type(y_test))

(740, 6) <class 'numpy.ndarray'>
(740, 3) <class 'numpy.ndarray'>
(186, 6) <class 'numpy.ndarray'>
(186, 3) <class 'numpy.ndarray'>


In [29]:
def lightgbm_multi_train(X_train: np.ndarray,
                         y_train: np.ndarray, 
                         params: dict = None):
    
    if params is None:
        params = {
            "objective": "regression",   
            "boosting_type": "gbdt",     
            "learning_rate": 0.05,       
            "num_leaves": 31,
            "max_depth": -1,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "n_jobs": -1,
            "random_state": 42
        }
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)
    base_estimator = LGBMRegressor(**params, n_estimators=1000)
    multi_model = MultiOutputRegressor(base_estimator)
    multi_model.fit(X_train, y_train,
                    # eval_set=[(X_val, y_val)],
                    # eval_metric="rmse",
                    # callbacks=[
                    # early_stopping(stopping_rounds=50),
                    # log_evaluation(period=100)
                    # ]
                    )

    return multi_model

In [30]:
model = lightgbm_multi_train(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000149 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 613
[LightGBM] [Info] Number of data points in the train set: 740, number of used features: 6
[LightGBM] [Info] Start training from score 0.428139
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [31]:
y_pred = model.predict(X_test)
y_pred.shape

(186, 3)

In [62]:
y_train.shape

(740, 3)

In [75]:
# for key, value in target_dict.items():
#     print(key)
#     print(value)

9
strength
0
cement
3
water


In [96]:
import numpy as np

def lightgbm_multi_evaluate(model, y_train, y_pred, y_test, target_cols):
    rmse_list, mae_list, r2_list = [], [], []
    
    for idx, col in enumerate(target_cols):
        y_true_i = y_test[:, idx]  # target_dict의 key를 인덱스로 변환
        y_pred_i = y_pred[:, idx]
        
        mae_i = np.mean(np.abs(y_true_i - y_pred_i))
        mse_i = np.mean((y_true_i - y_pred_i) ** 2)
        rmse_i = np.sqrt(mse_i)
        
        sse_i = np.sum((y_true_i - y_pred_i) ** 2)
        sst_i = np.sum(y_true_i - np.mean(y_train) ** 2)
        
        r2_i = 1 - sse_i / sst_i
        
        rmse_list.append(rmse_i)
        mae_list.append(mae_i)
        r2_list.append(r2_i)
        
        print(f"Target '{col}' - RMSE: {rmse_i:.4f}, MAE: {mae_i:.4f}, R2: {r2_i:.4f}")
    
    rmse_mean = np.mean(rmse_list)
    mae_mean = np.mean(mae_list)
    r2_mean = np.mean(r2_list)
    
    print(f"[Average Metrics] RMSE: {rmse_mean:.4f}, MAE: {mae_mean:.4f}, R2: {r2_mean:.4f}")
    return rmse_mean, mae_mean, r2_mean

In [97]:
rmse_mean, mae_mean, r2_mean = lightgbm_multi_evaluate(model,
                        y_train, 
                        y_pred,
                        y_test,
                        target_cols
                        )

print(f'lightGBM 모델 mulit-objective RMSE: {rmse_mean:.4f}')
print(f'lightGBM 모델 mulit-objective MAE: {mae_mean:.4f}')
print(f'lightGBM 모델 mulit-objective R^2: {r2_mean:.4f}')

Target 'strength' - RMSE: 0.0803, MAE: 0.0450, R2: 0.9674
Target 'cement' - RMSE: 0.0666, MAE: 0.0248, R2: 0.9755
Target 'water' - RMSE: 0.0545, MAE: 0.0207, R2: 0.9911
[Average Metrics] RMSE: 0.0672, MAE: 0.0302, R2: 0.9780
lightGBM 모델 mulit-objective RMSE: 0.0672
lightGBM 모델 mulit-objective MAE: 0.0302
lightGBM 모델 mulit-objective R^2: 0.9780


### catboost 단일 objective 구현

In [111]:
import src.datasets as datasets
import src.surrogate as surrogate
import src.search as search

In [112]:
load_data_func = getattr(datasets, f'cement_data')
X_train, X_test, y_train, y_test = load_data_func('./data/concrete_processed.csv')

In [113]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(740, 8)
(740, 1)
(186, 8)
(186, 1)


In [207]:
# 비교를 위한 선형회귀
from sklearn.linear_model import LinearRegression
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)
baseline_pred = baseline_model.predict(X_test)
baseline_r2 = 1 - np.sum((y_test - baseline_pred) ** 2) / (np.sum((y_test - y_train.mean()) ** 2) + 1e-10)
print(f'선형회귀 모델 R^2: {baseline_r2.item() :.4f}')

선형회귀 모델 R^2: 0.7556


In [209]:
from catboost import CatBoostRegressor

In [210]:
model = CatBoostRegressor(
    iterations=2000,        # 학습 반복 횟수
    depth=7,                # 트리 깊이
    learning_rate=0.05,     # 학습률
    bagging_temperature=1, # 앙상블 다양성을 조절 (1~3 추천)
    # l2_leaf_reg=5,         # L2 정규화 (3~10 사이에서 튜닝 가능)
    loss_function='RMSE',   # 손실 함수 (회귀 문제이므로 RMSE 사용)
    # eval_metric='RMSE',     # 평가 지표
    random_seed=42,
    verbose=100,            # 학습 과정 출력
    early_stopping_rounds=100  # 조기 종료
)

model.fit(X_train, y_train)

0:	learn: 0.2125008	total: 1.44ms	remaining: 2.88s
100:	learn: 0.0540607	total: 100ms	remaining: 1.88s
200:	learn: 0.0423818	total: 187ms	remaining: 1.67s
300:	learn: 0.0352422	total: 291ms	remaining: 1.64s
400:	learn: 0.0305735	total: 378ms	remaining: 1.51s
500:	learn: 0.0272640	total: 466ms	remaining: 1.39s
600:	learn: 0.0249423	total: 567ms	remaining: 1.32s
700:	learn: 0.0231242	total: 656ms	remaining: 1.22s
800:	learn: 0.0218541	total: 745ms	remaining: 1.11s
900:	learn: 0.0206984	total: 838ms	remaining: 1.02s
1000:	learn: 0.0198208	total: 926ms	remaining: 924ms
1100:	learn: 0.0191964	total: 1.01s	remaining: 828ms
1200:	learn: 0.0186435	total: 1.1s	remaining: 733ms
1300:	learn: 0.0181889	total: 1.19s	remaining: 639ms
1400:	learn: 0.0177914	total: 1.27s	remaining: 545ms
1500:	learn: 0.0174438	total: 1.36s	remaining: 454ms
1600:	learn: 0.0171564	total: 1.45s	remaining: 362ms
1700:	learn: 0.0169187	total: 1.54s	remaining: 271ms
1800:	learn: 0.0167320	total: 1.63s	remaining: 180ms
1900:

In [211]:
def eval_surrogate_model(y_train,y_pred, y_test):
    rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
    mae = np.mean(np.abs(y_test - y_pred))
    SSE = np.sum(np.square(y_test - y_pred))    
    SST = np.sum(np.square(y_test - y_train.mean()))
    r2 = 1 - SSE/SST
    return rmse, mae, r2

In [212]:
y_pred = model.predict(X_test)
print(y_pred.shape)
if y_pred.ndim == 1:
        y_pred = y_pred.reshape(-1, 1)
print(y_pred.shape)

(186,)
(186, 1)


In [213]:
rmse, mae, r2 = eval_surrogate_model(y_train, y_pred, y_test)
print(f'catboost 모델 RMSE: {rmse:.4f}')
print(f'catboost 모델 MAE: {mae:.4f}')
print(f'catboost 모델 R^2: {r2:.4f}')

catboost 모델 RMSE: 0.0496
catboost 모델 MAE: 0.0302
catboost 모델 R^2: 0.9430


### catboost으로 multiobjective 구현

In [14]:
import pandas as pd
df = pd.read_csv('./data/concrete_processed.csv')
df

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,0.089726,0.619702,0.000000,0.757426,0.000000,0.496512,0.422059,0.000000,0.357597
1,0.152740,0.123356,0.621189,0.309901,0.488688,0.813372,0.562353,-0.666667,0.274815
2,0.337900,0.000000,0.478261,0.598020,0.248869,0.453198,0.753529,0.000000,0.348904
3,0.374429,0.333236,0.000000,1.000000,0.000000,0.380814,0.191176,0.000000,0.564681
4,0.120548,0.536101,0.000000,0.656436,0.411765,0.716279,0.269706,0.000000,0.207084
...,...,...,...,...,...,...,...,...,...
921,0.089726,0.619702,0.000000,0.757426,0.000000,0.496512,0.422059,-1.000000,0.104580
922,0.445662,0.000000,0.587206,0.473267,0.429864,0.644767,0.436765,-1.190476,0.254055
923,0.075342,0.000000,0.829585,0.524752,0.452489,0.465116,0.588235,0.000000,0.142208
924,0.398174,0.339082,0.451274,0.520792,0.402715,0.200872,0.480294,0.000000,0.544310


In [15]:
target_cols = ['strength', 'cement', 'water']
X = df.drop(columns=target_cols)
y = df[target_cols]    

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

In [17]:
print(X_train.shape, type(X_train))
print(y_train.shape, type(y_train))
print(X_test.shape, type(X_test))
print(y_test.shape, type(y_test))

(740, 6) <class 'pandas.core.frame.DataFrame'>
(740, 3) <class 'pandas.core.frame.DataFrame'>
(186, 6) <class 'pandas.core.frame.DataFrame'>
(186, 3) <class 'pandas.core.frame.DataFrame'>


In [19]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_train.shape, type(X_train))
print(y_train.shape, type(y_train))
print(X_test.shape, type(X_test))
print(y_test.shape, type(y_test))

(740, 6) <class 'numpy.ndarray'>
(740, 3) <class 'numpy.ndarray'>
(186, 6) <class 'numpy.ndarray'>
(186, 3) <class 'numpy.ndarray'>


#### 방법 1 : 개별모델학습해서 multi objective 최적화

In [114]:
from catboost import CatBoostRegressor

In [115]:
models = []
for i in range(y.shape[1]):
    model = CatBoostRegressor(
        iterations=2000,
        depth=7,
        learning_rate=0.05,
        loss_function='RMSE',
        random_seed=42,
        verbose=200
    )
    model.fit(X_train, y_train[:, i], early_stopping_rounds=100)
    models.append(model)

0:	learn: 0.2125008	total: 1.26ms	remaining: 2.52s
200:	learn: 0.0423818	total: 197ms	remaining: 1.76s
400:	learn: 0.0305735	total: 378ms	remaining: 1.51s
600:	learn: 0.0249423	total: 557ms	remaining: 1.3s
800:	learn: 0.0218541	total: 738ms	remaining: 1.1s
1000:	learn: 0.0198208	total: 920ms	remaining: 918ms
1200:	learn: 0.0186435	total: 1.11s	remaining: 738ms
1400:	learn: 0.0177914	total: 1.3s	remaining: 556ms
1600:	learn: 0.0171564	total: 1.5s	remaining: 374ms
1800:	learn: 0.0167320	total: 1.7s	remaining: 188ms
1999:	learn: 0.0164236	total: 1.89s	remaining: 0us


IndexError: index 1 is out of bounds for axis 1 with size 1

In [99]:
y_preds = []
for m in models:
    y_pred = m.predict(X_test)
    print(y_pred.shape)
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(-1, 1)
    print(y_pred.shape)
    y_preds.append(y_pred)
y_preds = np.column_stack(y_preds)
print(y_preds.shape)

# y_preds = np.column_stack([m.predict(X_test) for m in models])

(186,)
(186, 1)
(186,)
(186, 1)
(186,)
(186, 1)
(186, 3)


In [100]:
import numpy as np

def catboost_multi_evaluate(model, y_train, y_pred, y_test, target_cols):
    rmse_list, mae_list, r2_list = [], [], []
    
    for idx, col in enumerate(target_cols):
        y_true_i = y_test[:, idx]  # target_dict의 key를 인덱스로 변환
        y_pred_i = y_pred[:, idx]
        
        mae_i = np.mean(np.abs(y_true_i - y_pred_i))
        mse_i = np.mean((y_true_i - y_pred_i) ** 2)
        rmse_i = np.sqrt(mse_i)
        
        sse_i = np.sum((y_true_i - y_pred_i) ** 2)
        sst_i = np.sum(y_true_i - np.mean(y_train) ** 2)
        
        r2_i = 1 - sse_i / sst_i
        
        rmse_list.append(rmse_i)
        mae_list.append(mae_i)
        r2_list.append(r2_i)
        
        print(f"Target '{col}' - RMSE: {rmse_i:.4f}, MAE: {mae_i:.4f}, R2: {r2_i:.4f}")
    
    rmse_mean = np.mean(rmse_list)
    mae_mean = np.mean(mae_list)
    r2_mean = np.mean(r2_list)
    
    print(f"[Average Metrics] RMSE: {rmse_mean:.4f}, MAE: {mae_mean:.4f}, R2: {r2_mean:.4f}")
    return rmse_mean, mae_mean, r2_mean

In [101]:
rmse_mean, mae_mean, r2_mean = catboost_multi_evaluate(model,
                            y_train,    
                            y_preds,  
                            y_test,   
                            target_cols)
print(f'catboost 모델 mulit-objective RMSE: {rmse_mean:.4f}')
print(f'catboost 모델 mulit-objective MAE: {mae_mean:.4f}')
print(f'catboost 모델 mulit-objective R^2: {r2_mean:.4f}')

Target 'strength' - RMSE: 0.0796, MAE: 0.0421, R2: 0.9680
Target 'cement' - RMSE: 0.0641, MAE: 0.0224, R2: 0.9773
Target 'water' - RMSE: 0.0584, MAE: 0.0219, R2: 0.9898
[Average Metrics] RMSE: 0.0674, MAE: 0.0288, R2: 0.9784
catboost 모델 mulit-objective RMSE: 0.0674
catboost 모델 mulit-objective MAE: 0.0288
catboost 모델 mulit-objective R^2: 0.9784


#### 방법 2 : custom loss function 활용 - 실패

In [224]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(740, 6)
(740, 3)
(186, 6)
(186, 3)


In [133]:
class MultiObjectiveLoss:
    def __init__(self, alpha=0.5, beta=0.5):
        """
        Multi-Objective 손실 함수 (RMSE + MAE)
        alpha: RMSE 가중치
        beta: MAE 가중치
        """
        self.alpha = alpha
        self.beta = beta
    
    def calc_ders_range(self, approxes, targets, weights):
        """
        approxes: 예측값 (logits)
        targets: 실제 값
        weights: 샘플 가중치 (필요 없을 경우 None)
        """
        assert len(approxes) == len(targets)
        
        derivatives = []
        second_derivatives = []
        
        for i in range(len(approxes)):
            error1 = approxes[i] - targets[i]  # RMSE 오차
            error2 = approxes[i] - targets[i]  # MAE 오차
            
            grad_rmse = error1 # RMSE 미분값
            grad_mae = np.sign(error2) # MAE 미분값
            
            grad = self.alpha * grad_rmse + self.beta * grad_mae  # Gradient (1차 미분)
            # Hessian (2차 미분) - 보통 1로 설정
            hess = 1.0  # CatBoost에서는 Hessian을 안 쓰는 경우가 많음
            
            derivatives.append(grad)
            second_derivatives.append(hess)
        
        return zip(derivatives, second_derivatives)

In [134]:
model = CatBoostRegressor(
    iterations=2000,
    depth=7,
    learning_rate=0.05,
    loss_function=MultiObjectiveLoss(alpha=0.7, beta=0.3),  # 가중치 조정 가능
    random_seed=42,
    verbose=200
)

In [135]:
model.fit(X_train, y_train)

CatBoostError: catboost/private/libs/target/data_providers.cpp:639: Currently only multi-regression, multilabel and survival objectives work with multidimensional target

#### 방법 3 :  CatBoost의 MultiRegression 모드

In [225]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(740, 6)
(740, 3)
(186, 6)
(186, 3)


In [116]:
model = CatBoostRegressor(
    iterations=2000,
    depth=7,
    learning_rate=0.05,
    loss_function="MultiRMSE",  # MultiRegression을 위한 손실 함수
    random_seed=42,
    verbose=200
)

In [117]:
model.fit(X_train, y_train)


0:	learn: 0.2127922	total: 1.05ms	remaining: 2.1s
200:	learn: 0.0406588	total: 193ms	remaining: 1.73s
400:	learn: 0.0292886	total: 377ms	remaining: 1.5s
600:	learn: 0.0244439	total: 560ms	remaining: 1.3s
800:	learn: 0.0218120	total: 745ms	remaining: 1.11s
1000:	learn: 0.0203332	total: 926ms	remaining: 924ms
1200:	learn: 0.0193429	total: 1.11s	remaining: 738ms
1400:	learn: 0.0186039	total: 1.29s	remaining: 553ms
1600:	learn: 0.0181165	total: 1.49s	remaining: 372ms
1800:	learn: 0.0177533	total: 1.7s	remaining: 188ms
1999:	learn: 0.0174305	total: 1.89s	remaining: 0us


In [118]:
y_preds = model.predict(X_test)
print(y_preds.shape)
print(type(y_preds))

(186,)
<class 'numpy.ndarray'>


In [119]:
print(y_test.shape)
print(type(y_test))

(186, 1)
<class 'numpy.ndarray'>


In [120]:
import numpy as np

def catboost_multi_evaluate(model, y_train, y_pred, y_test, target_cols):
    rmse_list, mae_list, r2_list = [], [], []
    
    for idx, col in enumerate(target_cols):
        y_true_i = y_test[:, idx]  # target_dict의 key를 인덱스로 변환
        y_pred_i = y_pred[:, idx]
        
        mae_i = np.mean(np.abs(y_true_i - y_pred_i))
        mse_i = np.mean((y_true_i - y_pred_i) ** 2)
        rmse_i = np.sqrt(mse_i)
        
        sse_i = np.sum((y_true_i - y_pred_i) ** 2)
        sst_i = np.sum(y_true_i - np.mean(y_train) ** 2)
        
        r2_i = 1 - sse_i / sst_i
        
        rmse_list.append(rmse_i)
        mae_list.append(mae_i)
        r2_list.append(r2_i)
        
        print(f"Target '{col}' - RMSE: {rmse_i:.4f}, MAE: {mae_i:.4f}, R2: {r2_i:.4f}")
    
    rmse_mean = np.mean(rmse_list)
    mae_mean = np.mean(mae_list)
    r2_mean = np.mean(r2_list)
    
    print(f"[Average Metrics] RMSE: {rmse_mean:.4f}, MAE: {mae_mean:.4f}, R2: {r2_mean:.4f}")
    return rmse_mean, mae_mean, r2_mean

In [121]:
rmse_mean, mae_mean, r2_mean = catboost_multi_evaluate(model,
                            y_train,    
                            y_preds,  
                            y_test,   
                            target_cols)
print(f'catboost 모델 mulit-objective RMSE: {rmse_mean:.4f}')
print(f'catboost 모델 mulit-objective MAE: {mae_mean:.4f}')
print(f'catboost 모델 mulit-objective R^2: {r2_mean:.4f}')

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

### TabPFN으로 multiobjective 구현

In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(740, 8)
(740, 1)
(186, 8)
(186, 1)


In [104]:
from tabpfn import TabPFNRegressor

In [105]:
models = []
for i in range(y_train.shape[1]):
    model = TabPFNRegressor()
    # model.fit(X_train, y_train.iloc[:, i])
    model.fit(X_train, y_train[:, i])
    models.append(model)

In [107]:
y_preds = []
for m in models:
    y_pred = m.predict(X_test)
    print(y_pred.shape)
    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(-1, 1)
    print(y_pred.shape)
    y_preds.append(y_pred)
y_preds = np.column_stack(y_preds)
print(y_preds.shape, type(y_preds))

(186,)
(186, 1)
(186,)
(186, 1)
(186,)
(186, 1)
(186, 3) <class 'numpy.ndarray'>


In [108]:
import numpy as np

def tabpfn_multi_evaluate(model, y_train, y_pred, y_test, target_cols):
    rmse_list, mae_list, r2_list = [], [], []
    
    for idx, col in enumerate(target_cols):
        y_true_i = y_test[:, idx]  # target_dict의 key를 인덱스로 변환
        y_pred_i = y_pred[:, idx]
        
        mae_i = np.mean(np.abs(y_true_i - y_pred_i))
        mse_i = np.mean((y_true_i - y_pred_i) ** 2)
        rmse_i = np.sqrt(mse_i)
        
        sse_i = np.sum((y_true_i - y_pred_i) ** 2)
        sst_i = np.sum(y_true_i - np.mean(y_train) ** 2)
        
        r2_i = 1 - sse_i / sst_i
        
        rmse_list.append(rmse_i)
        mae_list.append(mae_i)
        r2_list.append(r2_i)
        
        print(f"Target '{col}' - RMSE: {rmse_i:.4f}, MAE: {mae_i:.4f}, R2: {r2_i:.4f}")
    
    rmse_mean = np.mean(rmse_list)
    mae_mean = np.mean(mae_list)
    r2_mean = np.mean(r2_list)
    
    print(f"[Average Metrics] RMSE: {rmse_mean:.4f}, MAE: {mae_mean:.4f}, R2: {r2_mean:.4f}")
    return rmse_mean, mae_mean, r2_mean

In [109]:
rmse_mean, mae_mean, r2_mean = tabpfn_multi_evaluate(model,
                            y_train,    
                            y_preds,  
                            y_test,   
                            target_cols)
print(f'tabpfn 모델 mulit-objective RMSE: {rmse_mean:.4f}')
print(f'tabpfn 모델 mulit-objective MAE: {mae_mean:.4f}')
print(f'tabpfn 모델 mulit-objective R^2: {r2_mean:.4f}')

Target 'strength' - RMSE: 0.0878, MAE: 0.0455, R2: 0.9611
Target 'cement' - RMSE: 0.0604, MAE: 0.0140, R2: 0.9799
Target 'water' - RMSE: 0.0476, MAE: 0.0112, R2: 0.9932
[Average Metrics] RMSE: 0.0653, MAE: 0.0236, R2: 0.9781
tabpfn 모델 mulit-objective RMSE: 0.0653
tabpfn 모델 mulit-objective MAE: 0.0236
tabpfn 모델 mulit-objective R^2: 0.9781
